# Bases de datos vectoriales

## Codificar datos a embeddings

In [1]:
!pip install -U sentence-transformers
!pip install chromadb
!pip install openai

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [2]:
df = pd.read_csv('../Corpus/imdb_top_1000.csv')

In [4]:
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [5]:
df['text'] = df.apply(lambda x : x['Overview']+' '+x['Director']+' '+x['Star1']+' '+x['Star2']+' '+x['Star3']+' '+x['Star4'], axis=1)

In [6]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [7]:
embeddings = model.encode(df['text'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [8]:
df['embeddings'] = embeddings.tolist()

In [9]:
df['ids'] = df.index
df['ids'] = df['ids'].astype('str')

In [10]:
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,text,embeddings,ids
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Two imprisoned men bond over a number of years...,"[-0.1110253855586052, -0.009453012607991695, -...",0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...,"[-0.10081294178962708, -0.021370647475123405, ...",1
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",When the menace known as the Joker wreaks havo...,"[0.005191356875002384, 0.042379382997751236, -...",2
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",The early life and career of Vito Corleone in ...,"[-0.07282993197441101, 0.007155461236834526, -...",3
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...,"[-0.08513759821653366, 0.05780960991978645, -0...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,A young New York socialite becomes interested ...,"[0.028408115729689598, -0.02919202856719494, -...",995
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,Sprawling epic covering the life of a Texas ca...,"[0.024076659232378006, -0.0367053784430027, 0....",996
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","In Hawaii in 1941, a private is cruelly punish...","[-0.08005397766828537, 0.05756457522511482, -0...",997
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,Several survivors of a torpedoed merchant ship...,"[-0.0208076573908329, 0.05352266877889633, -0....",998


## Chroma

In [11]:
import chromadb
from chromadb.utils import embedding_functions

In [12]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key='xxxxxxxx',
    model_name = 'text-embedding-ada-002'
)

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = 'all-MiniLM-L6-v2')

In [13]:
# instanciamos un cliente 
chroma_client = chromadb.Client()
# para hacer la conexion peristente
client_persistent = chromadb.PersistentClient(path='./chroma.db')

In [16]:
# crear coleccion 
db = client_persistent.get_collection(name='imdb',embedding_function= sentence_transformer_ef)

In [ ]:
# carga de datos o insercion de datos
db.add(
    ids = df['ids'].tolist(), # parametro rederentr a los ids de la base de datos
    embeddings = df['embeddings'].tolist(), # parametro referente a los embeddings
    metadatas = df.drop(['ids', 'embeddings', 'text'], axis=1, inplace=False).to_dict('records') # parametro referente a los metadatos

    
)

In [17]:
db.peek(1)

{'ids': ['0'],
 'embeddings': [[-0.1110253855586052,
   -0.009453012607991695,
   -0.06168840080499649,
   -0.018297705799341202,
   -0.009833205491304398,
   0.06417807936668396,
   0.08203237503767014,
   -0.0490679033100605,
   0.06276258826255798,
   -0.03238987177610397,
   0.01261426042765379,
   0.039338234812021255,
   -0.025924397632479668,
   0.02284662425518036,
   -0.025803232565522194,
   -0.030675353482365608,
   -0.06829347461462021,
   0.03191167116165161,
   0.025044336915016174,
   0.07082389295101166,
   0.006209637504070997,
   -0.030562784522771835,
   -0.01686890795826912,
   -0.03708736225962639,
   0.004426794126629829,
   0.032468799501657486,
   -0.010899757035076618,
   -0.05131012201309204,
   -0.003628186183050275,
   0.0382995530962944,
   0.028951983898878098,
   0.014759071171283722,
   0.10134261101484299,
   0.04917679727077484,
   0.0685054212808609,
   0.021276891231536865,
   0.007060502655804157,
   0.028189919888973236,
   0.05037761107087135,
   

## Chroma embeddings

In [ ]:
db_no_embeddings = client_persistent.create_collection(name='movies_db_no_embeddings',embedding_function = sentence_transformer_ef)


In [26]:
db_no_embeddings.add(
    ids = df['ids'].tolist(), 
    documents= df['text'].tolist(),
    metadatas = df.drop(['ids', 'embeddings', 'text'], axis=1, inplace=False).to_dict('records')
    
)

In [27]:
db_no_embeddings.peek(1)

{'ids': ['0'],
 'embeddings': [[-0.11102534830570221,
   -0.009452982805669308,
   -0.06168835610151291,
   -0.0182977132499218,
   -0.009833162650465965,
   0.06417816132307053,
   0.08203231543302536,
   -0.04906787723302841,
   0.06276260316371918,
   -0.03238976374268532,
   0.012614269740879536,
   0.03933821991086006,
   -0.02592437155544758,
   0.02284662052989006,
   -0.025803284719586372,
   -0.030675340443849564,
   -0.06829359382390976,
   0.031911659985780716,
   0.025044331327080727,
   0.07082387059926987,
   0.0062096030451357365,
   -0.03056275099515915,
   -0.01686892844736576,
   -0.0370873287320137,
   0.004426770843565464,
   0.03246884047985077,
   -0.010899778455495834,
   -0.051310159265995026,
   -0.0036282367072999477,
   0.03829953074455261,
   0.028951948508620262,
   0.014759032055735588,
   0.1013425812125206,
   0.04917675629258156,
   0.06850545853376389,
   0.02127688378095627,
   0.007060464937239885,
   0.028189942240715027,
   0.05037754029035568,
   

In [28]:
# eliminar el primer documento de la coleccion
db_no_embeddings.delete(ids = ['0'])

In [29]:
db_no_embeddings.peek(1)

{'ids': ['1'],
 'embeddings': [[-0.10081302374601364,
   -0.02137051522731781,
   -0.05492264777421951,
   -0.02907075732946396,
   -0.09806850552558899,
   0.03351396694779396,
   0.033286888152360916,
   0.0022160769440233707,
   -0.0223348718136549,
   0.04345111921429634,
   0.08506180346012115,
   0.0005228392546996474,
   0.018331186845898628,
   0.003509313566610217,
   -0.04528844356536865,
   -0.02041180431842804,
   -0.03603034466505051,
   0.0033596137072890997,
   -0.002141389762982726,
   0.010022429749369621,
   -0.028846027329564095,
   -0.0999264195561409,
   0.012006587348878384,
   0.03996679559350014,
   -0.07681243866682053,
   0.059612859040498734,
   -0.011757124215364456,
   0.020119303837418556,
   -0.028658024966716766,
   -0.015758810564875603,
   -0.03031150996685028,
   0.013745659030973911,
   0.04075117036700249,
   0.04419570043683052,
   0.05254553630948067,
   -0.05740448459982872,
   0.04785593971610069,
   0.027237486094236374,
   0.07748442888259888,

### Chroma Query

In [30]:
results = db_no_embeddings.query(
 query_texts=['a history witch elves and a ring'],
 n_results=2   
)

In [31]:
results

{'ids': [['609', '158', '10']],
 'distances': [[1.0179615020751953, 1.070123314857483, 1.1383110284805298]],
 'metadatas': [[{'Certificate': 'UA',
    'Director': 'Peter Jackson',
    'Genre': 'Adventure, Fantasy',
    'Gross': '258,366,855',
    'IMDB_Rating': 7.8,
    'Meta_score': 66.0,
    'No_of_Votes': 601408,
    'Overview': 'The dwarves, along with Bilbo Baggins and Gandalf the Grey, continue their quest to reclaim Erebor, their homeland, from Smaug. Bilbo Baggins is in possession of a mysterious and magical ring.',
    'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMzU0NDY0NDEzNV5BMl5BanBnXkFtZTgwOTIxNDU1MDE@._V1_UX67_CR0,0,67,98_AL_.jpg',
    'Released_Year': '2013',
    'Runtime': '161 min',
    'Series_Title': 'The Hobbit: The Desolation of Smaug',
    'Star1': 'Ian McKellen',
    'Star2': 'Martin Freeman',
    'Star3': 'Richard Armitage',
    'Star4': 'Ken Stott'},
   {'Certificate': 'U',
    'Director': 'Hayao Miyazaki',
    'Genre': 'Animation, Adventure, Family

In [46]:
results['metadatas'][0][0]

{'Certificate': 'UA',
 'Director': 'Peter Jackson',
 'Genre': 'Adventure, Fantasy',
 'Gross': '258,366,855',
 'IMDB_Rating': 7.8,
 'Meta_score': 66.0,
 'No_of_Votes': 601408,
 'Overview': 'The dwarves, along with Bilbo Baggins and Gandalf the Grey, continue their quest to reclaim Erebor, their homeland, from Smaug. Bilbo Baggins is in possession of a mysterious and magical ring.',
 'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMzU0NDY0NDEzNV5BMl5BanBnXkFtZTgwOTIxNDU1MDE@._V1_UX67_CR0,0,67,98_AL_.jpg',
 'Released_Year': '2013',
 'Runtime': '161 min',
 'Series_Title': 'The Hobbit: The Desolation of Smaug',
 'Star1': 'Ian McKellen',
 'Star2': 'Martin Freeman',
 'Star3': 'Richard Armitage',
 'Star4': 'Ken Stott'}

### Where
1. Estructura

`
{
    "metadata_field": {
        <Operator>: <Value>
    }
}
`
2. Operadores

$eq - equal to (string, int, float)

$ne - not equal to (string, int, float)

$gt - greater than (int, float)

$gte - greater than or equal to (int, float)

$lt - less than (int, float)

$lte - less than or equal to (int, float)

In [ ]:
results_where = db_no_embeddings.query(
    query_texts=['a history with elves and a ring'],
    n_results=2,
    where = {
        'IMDB_Rating' : {
            "$gte":8
        }
    }
)



In [ ]:
results_where

In [ ]:
results_where = db_no_embeddings.query(
    query_texts=['a history with elves and a ring'],
    n_results=2,
    where ={ "$and":[ {
        'IMDB_Rating' : {
            "$gte":8
        }
    },
                      {
        'Released_Year' : {
            "$ne":'2001'
        }
    }] }
)



In [ ]:
results_where

### Cargar índice de Chroma previamente creado

In [ ]:
client_persistent_2 = chromadb.PersistentClient(path="/content/data_embeddings")

In [ ]:
db_2 = client_persistent_2.get_collection('movies_db_no_embeddigs')

In [ ]:
db_2.peek(1)

# Pinecone

In [ ]:
!pip install pinecone-client

In [ ]:
import pinecone
from getpass import getpass

In [ ]:
pincone_api = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [ ]:
pinecone.init(api_key=pincone_api, environment="______________")

In [ ]:
dimension_embeddings = len(df['embeddings'][0])
dimension_embeddings

In [ ]:
pinecone.create_index('movies-emebeddings',dimension=dimension_embeddings, metric='cosine')

In [ ]:
index = pinecone.Index('movies-emebeddings')

In [ ]:
df = df.fillna(' ')

In [ ]:
df

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size=64

for i in tqdm(range(0, len(df), batch_size)):

    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb, metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

In [ ]:
index.fetch(['0'])

In [ ]:
index.delete(ids=['0'])

In [ ]:
index.fetch(['0'])

### Pinecone query

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()

responses = index.query(vector=query_vector, top_k = 3, include_metadata=True)

In [ ]:
responses

### Filter

`
filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    }
`

The metadata filters can be combined with AND and OR:

$eq - Equal to (number, string, boolean)

$ne - Not equal to (number, string, boolean)

$gt - Greater than (number)

$gte - Greater than or equal to (number)

$lt - Less than (number)

$lte - Less than or equal to (number)

$in - In array (string or number)

$nin - Not in array (string or number)


In [ ]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()

responses = index.query(vector=query_vector, top_k = 3, include_metadata=True,
                        filter={
                            "Genre":{'$ne':"Animation, Adventure, Comedy"}
                        })

In [ ]:
responses

### Load Index

In [ ]:
pinecone.init(api_key=pincone_api, environment="_________")

In [ ]:
index_2 = pinecone.Index('movies-emebeddings')

In [ ]:
query = 'a history of an space journey'
query_vector = model.encode(query).tolist()

responses = index_2.query(
  vector=query_vector,
  top_k=3,
  include_metadata=True,
)


In [ ]:
responses